In [1]:
import numpy as np
import pandas as pd
import json
# from pandas.io.json import json_normalize
import requests
from datetime import datetime, timedelta
from ast import literal_eval
import dataloader
import time
import helper
import pyodbc

In [2]:
apikey = helper.get_apikey()

In [3]:
datetime.today()
fetchdate = datetime.strftime(datetime.today() - timedelta(days=1), '%m_%d_%Y')
fetchdate

'12_29_2022'

In [46]:
dataurl = f'https://api.themoviedb.org/3/configuration/languages?api_key={apikey}'
data = json.loads(requests.get(dataurl).text)
languages = pd.DataFrame.from_dict(data).rename(columns={'iso_639_1': 'code'})
custom = {"code": "varchar(10) PRIMARY KEY",}
dataloader.full_load_with_index(df=languages, tbl="Languages", custom=custom)

DataFrame full loaded to Table: Languages


In [77]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    # query = "select id from movieids"
    query = "select id from movieids where id not in (select id from moviesraw)"
    ids = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_52849/2405849386.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ids = pd.read_sql(query, conn)


In [ ]:
x=2
y=1

for chunk in np.array_split(ids['id'], 10000):
    session = requests.Session()
    df_movies = None 
    dfs=[]
    for i in chunk:
        print(f"chunk: {x}, Record: {y}") 
        dataurl = f'https://api.themoviedb.org/3/movie/{i}?api_key={apikey}'
        data = json.loads(session.get(dataurl, stream=False).text)
        df = pd.json_normalize(data)
        dfs.append(df)
        y+=1
    
    df_credits = pd.concat(dfs)

    
    
    # df_movies['vote_average'] = [format(i, '.3f') for i in df_movies['vote_average']]
    # df_movies['popularity'] = [format(i, '.3f') for i in df_movies['popularity']]
    
    # df_movies.to_csv('movies.csv', index=False)

            
    if x==1:   
        custom = {"id": "int PRIMARY KEY", "revenue": "bigint", "budget": "bigint"}
        dataloader.full_load_with_index(df=df_movies, tbl="MoviesRaw", custom=custom)
    else:
        # df_movies = pd.read_csv('movies.csv', engine='python')
        # print(df_movies.shape)
        # df_movies.dropna(subset=['id'], axis=0, inplace=True)
        # df_movies = df_movies.astype('object')
        dataloader.inc_load_with_index(df=df_movies, tbl="MoviesRaw")
    print(f"Loaded chunk: {x}")  
    x+=1
    session.close()
    del dfs
    del session
    

In [4]:
cstring = helper.get_connstring()
with pyodbc.connect(cstring) as conn:
    # query = "select id from movies order by id"
    query = "select id from movies m where m.id > (select max(movieid) from moviecast) order by m.id"
    ids = pd.read_sql(query, conn)

/var/folders/74/ndv7v28d3078l2cq22t43pkc0000gn/T/ipykernel_1691/2032837213.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ids = pd.read_sql(query, conn)


In [5]:

x=2
y=1
total_chunks = len(ids)//1000 + 1
for chunk in np.array_split(ids['id'], len(ids)//1000 + 1):
    session = requests.Session()
    df_movies = None 
    dfs_cast=[]
    dfs_crew=[]
    for i in chunk:
        dataurl = f'https://api.themoviedb.org/3/movie/{i}/credits?api_key={apikey}'
        data = json.loads(session.get(dataurl, stream=False).text)
        
        if data =={'success': False, 'status_code': 34, 'status_message': 'The resource you requested could not be found.'}:
            continue
        
        df = pd.json_normalize(data)
        if (df.empty):
            print('No data found')
            continue
        df_cast = (
            df[['id','cast']].assign(cast = lambda x: x['cast'].apply(lambda x: literal_eval(str(x))))
            .explode('cast')
            .dropna()
            .assign(castid = lambda x: x['cast'].apply(lambda y: y['id']),
                    # character = lambda x: x['cast'].apply(lambda y: y['character']),
                    )
            .rename(columns={'id':'movieid'})
            .drop('cast', axis=1)
            )
        dfs_cast.append(df_cast)
        
        if df_cast.empty:
            print('No data found')
            continue
        
        df_crew = (
            df[['id','crew']].assign(crew = lambda x: x['crew'].apply(lambda x: literal_eval(str(x))))
            .explode('crew')
            .dropna()
            .assign(crewid = lambda x: x['crew'].apply(lambda y: y['id']),
                    department = lambda x: x['crew'].apply(lambda y: y['department']),
                    )
            .rename(columns={'id':'movieid'})
            .drop('crew', axis=1)
            )
        dfs_crew.append(df_crew)
        
        print(f"chunk {x} of {total_chunks}, movie no. {y} received")
        y+=1
    
    df_cast = pd.concat(dfs_cast)
    df_crew = pd.concat(dfs_crew)
    df_cast= df_cast.dropna().drop_duplicates()
    df_crew= df_crew.dropna().drop_duplicates()
    
    if x==1:   
        custom = {"movieid": "int not null", "castid": "int not null"}
        dataloader.full_load_with_index(df=df_cast, tbl="MovieCast", custom=custom)
        custom = {"movieid": "int not null", "crewid": "int not null", "department":"nvarchar(255) not null"}
        dataloader.full_load_with_index(df=df_crew, tbl="MovieCrew", custom=custom)
    else:
        # df_movies = pd.read_csv('movies.csv', engine='python')
        # print(df_movies.shape)
        # df_movies.dropna(subset=['id'], axis=0, inplace=True)
        # df_movies = df_movies.astype('object')
        dataloader.inc_load_with_index(df=df_cast, tbl="MovieCast")
        dataloader.inc_load_with_index(df=df_crew, tbl="MovieCrew")
    x+=1
    
    session.close()
    del dfs_crew, dfs_cast
    del session

chunk 2 of 4, movie no. 1 received
chunk 2 of 4, movie no. 2 received
No data found
chunk 2 of 4, movie no. 3 received
chunk 2 of 4, movie no. 4 received
chunk 2 of 4, movie no. 5 received
chunk 2 of 4, movie no. 6 received
chunk 2 of 4, movie no. 7 received
chunk 2 of 4, movie no. 8 received
chunk 2 of 4, movie no. 9 received
chunk 2 of 4, movie no. 10 received
No data found
No data found
No data found
chunk 2 of 4, movie no. 11 received
chunk 2 of 4, movie no. 12 received
chunk 2 of 4, movie no. 13 received
No data found
chunk 2 of 4, movie no. 14 received
No data found
No data found
chunk 2 of 4, movie no. 15 received
No data found
No data found
No data found
chunk 2 of 4, movie no. 16 received
chunk 2 of 4, movie no. 17 received
No data found
chunk 2 of 4, movie no. 18 received
chunk 2 of 4, movie no. 19 received
No data found
No data found
No data found
No data found
No data found
No data found
No data found
No data found
No data found
No data found
No data found
chunk 2 of 4, mov

In [22]:
dataurl = f'https://api.themoviedb.org/3/movie/{4951}/credits?api_key={apikey}'
data = json.loads(session.get(dataurl, stream=False).text)
df = pd.json_normalize(data)

In [23]:
df

,id,cast,crew
0,4951,"[{'adult': False, 'gender': 1, 'id': 12041, 'k...","[{'adult': False, 'gender': 2, 'id': 1800, 'kn..."


In [10]:
df_crew

,movieid,crewid,department
0,4951,1800,Art
0,4951,7413,Camera
0,4951,16593,Editing
0,4951,20359,Costume & Make-Up
0,4951,21068,Sound
...,...,...,...
0,4951,2201655,Production
0,4951,2253986,Production
0,4951,2276145,Crew
0,4951,2498277,Art


In [38]:
df_cast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 0 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movieid    44 non-null     int64 
 1   castid     44 non-null     int64 
 2   character  44 non-null     object
dtypes: int64(2), object(1)
memory usage: 1.4+ KB


In [39]:
df_crew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 0
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movieid     81 non-null     int64 
 1   crewid      81 non-null     int64 
 2   department  81 non-null     object
dtypes: int64(2), object(1)
memory usage: 2.5+ KB
